In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd

In [2]:
# smae same 

chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()) , options=chrome_options)

driver.maximize_window()
wait = WebDriverWait(driver, 5)
driver.implicitly_wait(5)


In [3]:
# Creating lists 

link = ""
product_name = []
product_price = []
product_ratings = []
product_ratings_num = []
product_link = []

In [6]:
# for loop to iterate over each pages                         
for i in range(1,20):
    driver.get(f'https://www.amazon.com/s?i=movies-tv&rh=n%3A2649512011&fs=true&page={i}&qid=1726454955&ref=sr_pg_3')
    items = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "s-result-item s-asin")]')))
    
    for item in items:
        # Get the name of movie
        name = item.find_element(By.XPATH, './/span[@class="a-size-medium a-color-base a-text-normal"]')
        product_name.append(name.text)
        # print(product_name)

        # Get the price
        whole_price = item.find_elements(By.XPATH, './/span[@class="a-price-whole"]')
        fraction_price = item.find_elements(By.XPATH, './/span[@class="a-price-fraction"]')

        if whole_price != [] and fraction_price != []:
            price = '.'.join([whole_price[0].text, fraction_price[0].text])
        else:
            price = 0
        product_price.append(price)
        # print(product_price)


        # find ratings box
        ratings_box = item.find_elements(By.XPATH, './/div[@class="a-row a-size-small"]/span')

        # find ratings and ratings_num
        if ratings_box != []:
            ratings = ratings_box[0].get_attribute('aria-label')
        else:
            ratings = 0

        product_ratings.append(ratings)
        # print(product_ratings)

        # Get the link
        link = item.find_element(By.XPATH, './/a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]').get_attribute("href")
        product_link.append(link)
        # print(product_link)

driver.quit()

In [7]:
import os

# Create the directory if it doesn't exist
os.makedirs("Data", exist_ok=True)

# Store the data into a dataframe and save it as a CSV file
dict_df = {"Name of movie": product_name, "Price ( in $ )": product_price, "Rating stars": product_ratings, "Link to the movie": product_link}
df = pd.DataFrame(dict_df)
df.to_csv("Data/Movie.csv", index=False)